In [1]:
# Mount path for Google Colab
from google.colab import drive
drive.mount('/content/drive/')

root_path = '/content/drive/My Drive/comp90042-project1/project-data/'

Mounted at /content/drive/


In [2]:
# Installing libraries
!pip install torch torchvision transformers

     |████████████████████████████████| 2.3MB 27.2MB/s 
     |████████████████████████████████| 3.3MB 49.4MB/s 
     |████████████████████████████████| 901kB 45.5MB/s 


In [ ]:
# Inspect train data using pandas
import pandas as pd

train_df = pd.read_json(root_path + 'train.data.jsonl', lines=True)
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345
0,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...",None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated': False, 'te...","{'contributors': None, 'truncated

In [3]:
# Build a preprocessing function for PyTorch DataLoader
import json
import pandas as pd

import torch
from torch.utils.data import Dataset

from sklearn import preprocessing

from transformers import AutoTokenizer

class PreprocessDataset(Dataset):

    def __init__(self, datafile, labelfile, maxlen):
        # Define constants
        self.REPLY_ID = 'in_reply_to_status_id_str'
        self.TEXT = 'text'

        # Initialise label encoder
        self.le = preprocessing.LabelEncoder()

        # Store train dataset in pandas dataframe
        self.train_df = pd.read_json(datafile, lines=True)

        # Store labels as a list encoded with 0 and 1
        # If labels == None, then preprocessing is done for test dataset
        if labelfile != None:
            with open(labelfile, 'r') as f:
                load_labels = json.load(f)
                self.labels = self.le.fit_transform(list(load_labels.values())).tolist()
        else:
            self.labels = None
        
        # Initialise BERT tokenizer using AutoTokenizer
        self.bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

        # The max length of the tokens
        self.maxlen = maxlen
    
    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, index):
        # Start preprocessing train_df
        source = ''
        replies = ''
        row_tweet = list(self.train_df.iloc[index])

        for tweet in row_tweet:
            if tweet == None:
                break
            if tweet[self.REPLY_ID] == None:
                source = tweet[self.TEXT]
            else:
                replies += tweet[self.TEXT] + ' '
        
        bert_tokens = self.bert_tokenizer(source,
                                          replies,
                                          padding='max_length',
                                          truncation=True,
                                          max_length=self.maxlen)
        
        # Example output of tokens
        # {'input_ids': [101, 138, 18696, 155, 1942, 3190, 1144, 1572, 13745, 1104, 159, 9664, 2107, 102],
        # 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        # 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
        
        if self.labels != None:
            label = self.labels[index]
        else:
            label = None

        tokens_ids_t = torch.tensor(bert_tokens['input_ids'])
        attn_mask_t = torch.tensor(bert_tokens['attention_mask'])
        seg_ids_t = torch.tensor(bert_tokens['token_type_ids'])

        if label == None:
            return tokens_ids_t, attn_mask_t, seg_ids_t

        return tokens_ids_t, attn_mask_t, seg_ids_t, label


In [4]:
from torch.utils.data import DataLoader

MAX_LEN = 512
BATCH_SIZE = 8
NUM_WORKERS = 2
SHUFFLE = True

# creating instances of training and development set
train_set = PreprocessDataset(datafile=root_path + 'train.data.jsonl',
                              labelfile=root_path + 'train.label.json',
                              maxlen=MAX_LEN)
dev_set = PreprocessDataset(datafile=root_path + 'dev.data.jsonl',
                            labelfile=root_path + 'dev.label.json',
                            maxlen=MAX_LEN)

# creating instances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE, num_workers=NUM_WORKERS)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE, num_workers=NUM_WORKERS)

print('Done preprocessing')


Done preprocessing


In [5]:
import torch
import torch.nn as nn
from transformers import BertModel

class RumourClassifier(nn.Module):

    def __init__(self):
        super(RumourClassifier, self).__init__()

        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')

        # classification layer
        self.cls_layer = nn.Linear(768, 1)
    
    def forward(self, seq, attn_masks, seg_ids):
        outputs = self.bert_layer(seq,
                                  attention_mask=attn_masks,
                                  token_type_ids=seg_ids)
        cont_reps = outputs.last_hidden_state

        cls_rep = cont_reps[:, 0]

        logits = self.cls_layer(cls_rep)

        return logits

In [6]:
gpu = 0

print("Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...")
net = RumourClassifier()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the rumour classifier.")

Creating the rumour classifier, initialised with pretrained BERT-BASE parameters...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the rumour classifier.


In [7]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [8]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, seg_ids, labels in dataloader:
            seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks, seg_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [ ]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        for it, (seq, attn_masks, seg_ids, labels) in enumerate(train_loader):
            # clear gradients
            opti.zero_grad()
            # converting these to cuda tensors
            seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks, seg_ids)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
            
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}, saving model...".format(ep, dev_acc, dev_loss))
        torch.save(net.state_dict(), root_path + 'ica_rumor_cls_baseline_{}.dat'.format(ep))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}".format(best_acc, dev_acc))
            best_acc = dev_acc

In [ ]:
num_epoch = 4

train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.8055034875869751; Accuracy: 0.0; Time taken (s): 1.155609130859375
Iteration 100 of epoch 0 complete. Loss: 0.26082682609558105; Accuracy: 0.875; Time taken (s): 71.69718074798584
Iteration 200 of epoch 0 complete. Loss: 0.10262247920036316; Accuracy: 1.0; Time taken (s): 75.97746276855469
Iteration 300 of epoch 0 complete. Loss: 0.20133917033672333; Accuracy: 0.875; Time taken (s): 79.11009478569031
Iteration 400 of epoch 0 complete. Loss: 0.1773877888917923; Accuracy: 1.0; Time taken (s): 81.25656604766846
Iteration 500 of epoch 0 complete. Loss: 0.1788734495639801; Accuracy: 0.875; Time taken (s): 81.86097812652588
Epoch 0 complete! Development Accuracy: 0.8698630332946777; Development Loss: 0.2825256723349225, saving model...
Best development accuracy improved from 0 to 0.8698630332946777
Iteration 0 of epoch 1 complete. Loss: 0.3012177646160126; Accuracy: 0.875; Time taken (s): 89.37632822990417
Iteration 100 of epoch 1 complete. Loss: 0.19

In [9]:
net.load_state_dict(torch.load(root_path + 'ica_rumor_cls_baseline_1.dat'))

<All keys matched successfully>

In [10]:
import json

def generate_prediction(net, dataset, maxlen):

    net.eval()

    REPLY_ID = 'in_reply_to_status_id_str'
    ID = 'id_str'

    predictions = dict()
    id_list = []
    preds = []

    test_df = pd.read_json(dataset, lines=True)

    test_set = PreprocessDataset(datafile=dataset,
                                 labelfile=None,
                                 maxlen=maxlen)
    
    test_loader = DataLoader(test_set)

    with torch.no_grad():
        for seq, attn_masks, seg_ids in test_loader:
            seq, attn_masks, seg_ids = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu)
            logits = net(seq, attn_masks, seg_ids)
            probs = torch.sigmoid(logits)
            soft_prob = (probs > 0.5).long()
            decoded_pred = 'non-rumour' if soft_prob.squeeze().item() == 0 else 'rumour'
            preds.append(decoded_pred)
    
    for index, row in test_df.iterrows():
        for tweet in row:
            if tweet[REPLY_ID] == None:
                id_list.append(tweet[ID])
                break
    
    for id, pred in zip(id_list, preds):
        predictions[id] = pred
    
    with open(root_path + 'test-output.json', 'w') as outfile:
        json.dump(predictions, outfile)
    
    print('predictions generated!!')

In [11]:
MAX_LEN = 512

generate_prediction(net, root_path + 'test.data.jsonl', MAX_LEN)

predictions generated!!


-------------------------------------
# This is for testing

In [ ]:
counter = 0
for it, (seq, attn_masks, seg_ids, labels) in enumerate(train_loader):
    if counter > 5:
        break
    counter += 1
    print(it, (seq, attn_masks, seg_ids, labels))

In [ ]:
MAX_LEN = 512


# creating instances of training and development set
test_set = PreprocessDataset(datafile=root_path + 'test.data.jsonl',
                             labelfile=None,
                              maxlen=MAX_LEN)

# creating instances of training and development dataloaders
test_loader = DataLoader(train_set)

with torch.no_grad():
    for seq, attn_masks, seg_ids, labels in test_loader:
        seq, attn_masks, seg_ids, labels = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids.cuda(gpu), labels.cuda(gpu)
        logits = net(seq, attn_masks, seg_ids)
        probs = torch.sigmoid(logits)
        soft_prob = (probs > 0.5).long()
        decoded_pred = soft_prob.squeeze().item()
        print(decoded_pred)


# prediction = net()

In [ ]:
# Preprocessing attempt
import json
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define constant
REPLY_ID = 'in_reply_to_status_id_str'
# ID = 'id_str'
TEXT = 'text'
# TIMESTAMP = 'created_at'
MAX_LEN = 512

count = 0

bert_X = []

for index, row in train_df.iterrows():
  source_bool = False
  source_tweet = ''
  reply_tweets = ''

  for tweet in row:
    if tweet == None:
      break
    if tweet[REPLY_ID] == None:
      source_tweet = tweet[TEXT]
    else:
      reply_tweets += tweet[TEXT] + ' '
  
  bert_tokens = bert_tokenizer(source_tweet, reply_tweets, padding=True, truncation=True, max_length=512)
  bert_X.append(bert_tokens)
  # print(bert_tokens)
  # count += 1
  # if count > 2:
  #   break

In [ ]:
# Testing bert
bert_model = BertModel.from_pretrained('bert-base-uncased')

outputs = bert_model(train_set.__getitem__(2)[0],
                     attention_mask=train_set.__getitem__(2)[1],
                     token_type_ids=train_set.__getitem__(2)[2])
hidden_reps = outputs.last_hidden_state
print(hidden_reps.shape)

torch.Size([1, 458, 768])


In [ ]:
import json
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

senc = ['this is a sen', 'this is another', 'and another one']
# print(senc)
senc_tokens = bert_tokenizer(senc)
print(senc_tokens)

{'input_ids': [[101, 2023, 2003, 1037, 12411, 102], [101, 2023, 2003, 2178, 102], [101, 1998, 2178, 2028, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}


In [ ]:
import json
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

REPLY_ID = 'in_reply_to_status_id_str'
ID = 'id_str'

train_id = []

for index, row in train_df.iterrows():
  for tweet in row:
    if tweet[REPLY_ID] == None:
      train_id.append(tweet[ID])
      break
  
with open(root_path + 'train.label.json', 'r') as f:
  train_labels = json.load(f)
  label_df = pd.DataFrame.from_dict(train_labels, orient='index', columns=['label'])
  label_le = le.fit_transform(label_df)
#   label_id = le.fit_transform(list(train_labels.values())).tolist()
#   label_df = pd.DataFrame(label_id)

print(label_le)
# print(label_id)
print(le.classes_)
print(label_df)

def Diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

# print(Diff(train_id, label_id))

[0 0 0 ... 1 0 0]
['non-rumour' 'rumour']
                         label
552800070199148544  non-rumour
544388259359387648  non-rumour
552805970536333314  non-rumour
525071376084791297      rumour
498355319979143168  non-rumour
...                        ...
524959027516932096      rumour
524940940721418240  non-rumour
580331453889708032      rumour
552820384039706624  non-rumour
499426471300329472  non-rumour

[4641 rows x 1 columns]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
index_test = []

for index, row in train_df.iterrows():
  source_index = 0
  for tweet in row:
    source_index += 1
    if tweet[REPLY_ID] == None:
      index_test.append(source_index)
      break

print(index_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 